In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import sys
import seaborn as sns
TRAIN_DATASET_PATH = 'train.csv'
TEST_DATASET_PATH = 'test.csv'

In [2]:
df = pd.read_csv(TRAIN_DATASET_PATH)
display(df.head())
print(df.shape)

,Id,age,years_of_experience,lesson_price,qualification,physics,chemistry,biology,english,geography,history,mean_exam_points,choose
0,0,35.0,0.0,2150.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,74.0,0
1,1,52.0,2.0,1250.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,57.0,1
2,2,29.0,3.0,1750.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,66.0,0
3,3,33.0,3.0,1050.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,66.0,1
4,4,46.0,3.0,2250.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,73.0,0


(10000, 13)


In [3]:
df_test = pd.read_csv(TEST_DATASET_PATH)
df['inter']=1
df_test['inter']=1

In [4]:
TARGET_NAME = 'choose'
FEATURE_NAMES = ['inter', 'age', 'years_of_experience', 'lesson_price', 'qualification',
       'physics', 'chemistry', 'biology', 'english', 'geography', 'history', 'mean_exam_points']
X = df[FEATURE_NAMES].values
Y = df[TARGET_NAME].values
index = df['Id'].values
X_test_result = df_test[FEATURE_NAMES].values

print(X.shape, Y.shape)

(10000, 12) (10000,)


In [5]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [6]:
def loss_function(x):
    return 0 if x > 0 else 1

In [7]:
def exp_loss_func(x):
    return np.exp(-x)
def square_loss(x):
    return (1 - x) ** 2
def logistic_loss(x):
    return np.log2(1 + np.exp(-x))

In [8]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [9]:
# перемешивание датасета
np.random.seed(42)
shuffle_index = np.random.permutation(index.shape[0])
X_shuffled, y_shuffled = X[shuffle_index], Y[shuffle_index]

# разбивка на обучающую и тестовую выборки
train_proportion = 0.7
train_test_cut = int(len(X) * train_proportion)

X_train, X_test, y_train, y_test = \
    X_shuffled[:train_test_cut], \
    X_shuffled[train_test_cut:], \
    y_shuffled[:train_test_cut], \
    y_shuffled[train_test_cut:]
    
print("Размер массива признаков обучающей выборки", X_train.shape)
print("Размер массива признаков тестовой выборки", X_test.shape)
print("Размер массива ответов для обучающей выборки", y_train.shape)
print("Размер массива ответов для тестовой выборки", y_test.shape)

Размер массива признаков обучающей выборки (7000, 12)
Размер массива признаков тестовой выборки (3000, 12)
Размер массива ответов для обучающей выборки (7000,)
Размер массива ответов для тестовой выборки (3000,)


In [10]:
X_train_tr = X_train.transpose()
y_train_tr = y_train.reshape(1, y_train.shape[0])
X_test_tr = X_test.transpose()
y_test_tr = y_test.reshape(1, y_test.shape[0])
X_train_tr

array([[ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [59., 46., 53., ..., 43., 51., 35.],
       [ 0.,  0.,  1., ...,  0.,  0.,  1.],
       ...,
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [85., 53., 52., ..., 79., 60., 54.]])

In [11]:
X_st = X_train_tr.copy()
X_st[1, :] = calc_std_feat(X_train_tr[1, :])
X_st[3, :] = calc_std_feat(X_train_tr[3, :])
X_st[11, :] = calc_std_feat(X_train_tr[11, :])
X_test_st = X_test_tr.copy()
X_test_st[1, :] = calc_std_feat(X_test_tr[1, :])
X_test_st[3, :] = calc_std_feat(X_test_tr[3, :])
X_test_st[11, :] = calc_std_feat(X_test_tr[11, :])

In [12]:
def log_loss(w, X, y):
    m = X.shape[1]
    # используем функцию сигмоиды, написанную ранее
    A = sigmoid(np.dot(w.T, X))    
    loss = -1.0 / m * np.sum(y * np.log(A) + (1 - y) * np.log(1 - A))
    loss = np.squeeze(loss)
    grad = 1.0 / m * np.dot(X, (A - y).T)    
    return loss, grad

In [13]:
def optimize(w, X, y, n_iterations, eta):
#     потери будем записывать в список для отображения в виде графика
    losses = []   
    for i in range(n_iterations):        
        loss, grad = log_loss(w, X, y)
        w = w - eta * grad
        losses.append(loss)        
    return w, losses

In [14]:
def predict(w, X):    
    m = X.shape[1] 
    y_predicted = np.zeros((1, m))
    w = w.reshape(X.shape[0], 1)
    A = sigmoid(np.dot(w.T, X))  
#     За порог отнесения к тому или иному классу примем вероятность 0.5
#    for i in range(A.shape[1]):
#        if (A[:,i] > 0.5): 
#            y_predicted[:, i] = 1
#        elif (A[:,i] <= 0.5):
#            y_predicted[:, i] = 0    
    for i in range(A.shape[1]):
        y_predicted[:, i] = A[:,i]
    return y_predicted

In [15]:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred + 1e-12) + (1.0 - y) * np.log(1.0 - y_pred - 1e-12))
    return err
def eval_model(X, y, iterations, alpha=1e-4, lambda_=1e-8):
    np.random.seed(13)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * 2 * np.dot((y_pred - y), X.T) + lambda_ * W) 
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [16]:
# иницилизируем начальный вектор весов
w0 = np.zeros((X_st.shape[0], 1))
n_iterations = 1000
eta = 0.05
w, losses = optimize(w0, X_st, y_train_tr, n_iterations, eta)
y_predicted_test = predict(w, X_test_st)
y_predicted_train = predict(w, X_st)
# В качестве меры точности возьмем долю правильных ответов
train_accuracy = 100.0 - np.mean(np.abs(y_predicted_train - y_train_tr)*100.0)
test_accuracy = 100.0 - np.mean(np.abs(y_predicted_test-y_test_tr)*100.0)
print(f"Итоговый вектор весов w: {w}")
print(f"Точность на обучающей выборке: {train_accuracy:.3f}")
print(f"Точность на тестовой выборке: {test_accuracy:.3f}")

Итоговый вектор весов w: [[-1.29378589e+00]
 [ 3.22823036e-02]
 [-4.14380986e-02]
 [-7.34722718e-01]
 [-5.81716035e-01]
 [ 3.25211399e-01]
 [ 2.06142270e-01]
 [ 1.99518951e-01]
 [ 1.74923983e-04]
 [ 2.65780982e-03]
 [-4.85705829e-02]
 [ 1.20786028e+00]]
Точность на обучающей выборке: 81.272
Точность на тестовой выборке: 80.765


In [17]:
X_test_result = X_test_result.transpose()
X_test_result_st = X_test_result.copy()
X_test_result_st[1, :] = calc_std_feat(X_test_result[1, :])
X_test_result_st[3, :] = calc_std_feat(X_test_result[3, :])
X_test_result_st[11, :] = calc_std_feat(X_test_result[11, :])

In [18]:
#y_result = predict(w, X_test_result_st)

In [19]:
#submissions = pd.concat([df_test['Id'], pd.Series(y_result[0])], axis=1)

In [20]:
#submissions = submissions.rename(columns={0: 'choose'})

In [21]:
#submissions.to_csv('Kalmina_classif_log.csv',index=None)

In [22]:
W = eval_model(X_st, y_train_tr[0], iterations=500000, alpha=0.015)

50000 [-6.31143367  0.02311878 -0.04091199 -2.89982365  1.56962745  0.91712293
  0.82595176  0.79399108  0.34910965  0.28899744 -0.31200464  1.59661598] 0.24818496957798125
100000 [-6.51074395  0.02308624 -0.04129993 -2.97738591  1.64895903  0.94438214
  0.84502024  0.81367409  0.36497753  0.34707359 -0.14533591  1.60423438] 0.2481034820638519
150000 [-6.5246305   0.02306773 -0.04128928 -2.98265411  1.65433224  0.94612292
  0.84626221  0.81514822  0.36588188  0.35024221 -0.10751512  1.6047252 ] 0.24810207678392018
200000 [-6.52589647  0.023063   -0.04128122 -2.98310224  1.65479703  0.94625809
  0.8463595   0.81530143  0.36592886  0.35051641 -0.09972308  1.60475247] 0.24810202600137893
250000 [-6.5260594   0.02306199 -0.04127926 -2.98315564  1.65485358  0.94627248
  0.84636994  0.81532362  0.36593033  0.35055243 -0.09815718  1.60475362] 0.24810202399231576
300000 [-6.52608653  0.02306178 -0.04127885 -2.98316414  1.65486269  0.9462746
  0.84637148  0.81532754  0.36593015  0.35055854 -0.0

In [23]:
y_result_two = predict(W, X_test_result_st)

In [24]:
submissions_two = pd.concat([df_test['Id'], pd.Series(y_result_two[0])], axis=1)
submissions_two = submissions_two.rename(columns={0: 'choose'})
submissions_two.to_csv('Kalmina_classif_log_two_test.csv',index=None)

In [25]:
y_result_two

array([[0.013436  , 0.18846915, 0.01224849, ..., 0.32036361, 0.90916409,
        0.02031071]])